In [1]:
import mlflow
from mlflow import MlflowClient
from mlflow.entities import ViewType


In [2]:
MLFLOW_TRACKING_URI = 'http://54.163.199.103:5000/'
EXPERIMENT_NAME = 'portugal-rent-price'

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [4]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
query = 'tags.runtype = "hyperparameter_tuning"'

In [8]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    filter_string='tags.runtype = "best_model"',
    order_by=["metrics.rmse ASC"],
)[0]

best_model_rmse = best_run.data.metrics['rmse']
run_id = best_run.info.run_id

In [9]:
model_tunnings = client.search_runs(
            experiment_ids=["1"],
            filter_string=query,
            order_by=["metrics.rmse ASC", "metrics.r2 DESC"],
            max_results=5,
        )

In [10]:
model_tunnings[0]

<Run: data=<RunData: metrics={'r2': 0.6924328765364193, 'rmse': 1083.7872154557951}, params={'learning_rate': '0.27940914581222503',
 'max_depth': '5',
 'min_child_weight': '1.128011907112829',
 'reg_alpha': '0.040383536932105425',
 'reg_lambda': '0.5896272832277738',
 'seed': '42'}, tags={'mlflow.runName': 'tasteful-carp-428',
 'mlflow.source.git.commit': 'aa663eaf013a61d3299fdda589232a7b6bb7f0c0',
 'mlflow.source.name': 'E:\\Estudos\\Projetos\\Python\\mlops_final_project\\.venv\\lib\\site-packages\\prefect\\engine.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'MarcosPaulo',
 'model': 'xboostRegressor',
 'runtype': 'hyperparameter_tuning'}>, info=<RunInfo: artifact_uri='s3://final-project-mlops/mlflow/1/9d678377214149e394e267941e634f92/artifacts', end_time=1691010202801, experiment_id='1', lifecycle_stage='active', run_id='9d678377214149e394e267941e634f92', run_name='tasteful-carp-428', run_uuid='9d678377214149e394e267941e634f92', start_time=1691010201386, status='FINISHED', us

In [11]:
best_param = model_tunnings[0].data.params

In [12]:
best_param

{'learning_rate': '0.27940914581222503',
 'max_depth': '5',
 'min_child_weight': '1.128011907112829',
 'reg_alpha': '0.040383536932105425',
 'reg_lambda': '0.5896272832277738',
 'seed': '42'}

In [13]:
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
query = 'tags.runtype = "best_model"'
best_run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    filter_string=query,
    order_by=["metrics.rmse ASC"],
)[0]

In [16]:
best_run.info.run_id

'9990c3108cd147b88db15cc07d4b7ace'

In [35]:
latest_prod_version = client.get_latest_versions(
    EXPERIMENT_NAME, stages=["Production"]
)


In [40]:
latest_prod_version[0].run_id

'64ffbb9362c64ca2b545e04665b26075'

In [48]:
prod_model = latest_prod_version[0]

In [52]:
latest_prod_version[0].run_id

'64ffbb9362c64ca2b545e04665b26075'

In [51]:
prod_model = client.get_run(prod_model.run_id)
prod_model.data.metrics['rmse']

1083.7872154557951

1083.7872154557951

In [26]:
model_name = EXPERIMENT_NAME
latest_versions = client.get_latest_versions(model_name)

In [27]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    print(version)


version: 1, stage: Production
<ModelVersion: aliases=[], creation_timestamp=1690975772499, current_stage='Production', description='The model version 1 was transitioned to Production on 2023-08-02', last_updated_timestamp=1690975773469, name='portugal-rent-price', run_id='64ffbb9362c64ca2b545e04665b26075', run_link='', source='s3://final-project-mlops/mlflow/1/64ffbb9362c64ca2b545e04665b26075/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>
version: 3, stage: None
<ModelVersion: aliases=[], creation_timestamp=1691010904466, current_stage='None', description='', last_updated_timestamp=1691010904466, name='portugal-rent-price', run_id='9990c3108cd147b88db15cc07d4b7ace', run_link='', source='s3://final-project-mlops/mlflow/1/9990c3108cd147b88db15cc07d4b7ace/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>


In [28]:
model_prod_id = [version.run_id for version in latest_versions if version.current_stage == "Production"]

In [29]:
model_prod_id

['64ffbb9362c64ca2b545e04665b26075']

In [30]:
prod_model = client.get_run(model_prod_id[0])

In [31]:
prod_model.data.metrics['rmse']

1083.7872154557951